1. Add the following Python Libraries to the Databricks Workspace
    <code>
    Click on "Workspace" > "Shared" > "Create Library" > "Upload Python Egg or PyPi" <br>
     azure-cognitiveservices-language-luis <br>
     azure-cognitiveservices-language-textanalytics <br>
     SpeechRecognition <br>
    </code>
    Select "Automatically attach to all clusters"
2. Initialize a Custom Vision Service by signing into https://customvision.ai/<br>
    Click on the "Settings" Icon and copy the TRAINING KEY and PREDICTION KEY <br>

## Using the Text Analyics Cognitive Service
The previous examples demonstrate some ways to write code and analyze text, and they serve to illustrate that text analytics involves applying statistical techniques to text data in order to discern semantic meaning. This is a common theme in many AI solutions.

Microsoft Cognitive Services includes a Text Analytics service that encapsulates much more sophisticated techniques for ascertaining meaning from text.

### Create a Text Analytics Service
To see this in action, you need to provision a Text Analytics service in your Azure subscription. Follow these steps to do that:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ New**, and in the **AI + Cognitive Services** category, click **See all**.
4. In the list of cognitive services, click **Text Analytics API**.
5. In the **Text Analytics API** blade, click **Create**.
6. In the **Create** blade, enter the following details, and then click **Create**
  * **Name**: A unique name for your service.
  * **Subscription**: Your Azure subscription.
  * **Location**: Choose the Azure datacenter location where you want to host your service.
  * **Pricing tier**: Choose the F0 pricing tier.
  * **Resource Group**: Choose the existing resource group you created in the previous lab (or create a new one if you didn't complete the previous lab)
  * Read the notice about the use of your data, and select the checkbox.
7. Wait for the service to be created.
8. When deployment is complete, click **All Resources** and then click your Text Analytics service to open its blade.
9. In the blade for your Text Analytics service, note the **Endpoint** URL. Then assign the base URI (*location*.api.cognitive.microsoft.com) for your service to the **textAnalyticsURI** variable in the cell below.
10. In the blade for your Text Analytics service, click **Keys** and then copy **Key 1** to the clipboard and paste it into the **textKey** variable assignment value in the cell below. 
11. Run the cell below to assign the variables.

In [4]:
textAnalyticsURI = 'eastus2.api.cognitive.microsoft.com'
textKey = '<INSERT TEXT ANALYTICS API KEY HERE>'

### Call the Text Analytics Service to Determine Key Phrases in the Documents
One of the methods provided by the Text Analytics service is the ability to extract a list of key phrases from text documents, which give an insight into the core topics discussed in the document.

Run the following cell to call the **keyPhrases** method of the Text Analytics service and extract the key phrases for the text documents you have loaded so far in this notebook.

In [6]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

# Define the request headers.
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': textKey,
    'Accept': 'application/json'
}

# Define the parameters
params = urllib.parse.urlencode({
})

# Define the request body
body = {
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": "The company uses machine learning to analyze data collected from behavior on its website, and other information shared voluntarily. TrueAccord says it does not buy any personal, financial, or demographic data, including credit scores, does not use affinity data, and does not 'creep crawl the web.' But it does know how much a debtor owes, to whom, and how far behind the person is on the payments. Over time, the company believes this data will help it predict preferences, like whether customers prefer text versus email, days and times to send messages, and even tone of voice, such as empathetic, friendly, or inspirational, but never aggressive."
    },
    {
          "language": "en",
          "id": "2",
          "text": "The breakthrough effort was backed, in part, by a startup called Exonics, which was cofounded in 2017 by Olson and patient advocacy group CureDuchenne. Headquartered in Cambridge, Exonics has licensed the gene editing technology developed by Olson's lab and is moving it toward human trials, with the hopes of one day commercializing treatments. The young biotech company got its footing with $2 million from CureDuchenne's venture arm, and it has since raised more than $40 million from The Column Group."
    }
  ]
}

try:
    # Execute the REST API call and get the response.
    conn = http.client.HTTPSConnection(textAnalyticsURI)
    conn.request("POST", "/text/analytics/v2.0/keyPhrases?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read().decode("UTF-8")

    # 'data' contains the JSON data. The following formats the JSON data for display.
    parsed = json.loads(data)
    for document in parsed['documents']:
        print("Document " + document["id"] + " key phrases:")
        for phrase in document['keyPhrases']:
            print("  " + phrase)
        print("---------------------------")
    conn.close()

except Exception as e:
    print('Error:')
    print(e)

Document 1 key phrases:
 demographic data
 affinity data
 company
 text
 days
 credit scores
 messages
 customers
 times
 email
 tone of voice
 website
 preferences
 creep
 behavior
 machine
 person
 payments
 information
 TrueAccord
 debtor
---------------------------
Document 2 key phrases:
 Exonics
 Column Group
 patient advocacy group CureDuchenne
 gene editing technology
 hopes
 footing
 Olson's lab
 day commercializing treatments
 young biotech company
 CureDuchenne's venture arm
 human trials
 Cambridge
 breakthrough effort
 startup
---------------------------

From these key phrases, it's reasonably clear that the first document is about freedom and nationhood, while the second is about software services for AI.

### Perform Sentiment Analysis
Another common AI requirement is to determine the sentiment associated with some text. For example, you might analyze tweets that include your organization's twitter handle to determine if they are positive or negative.

Run the cell below to use the **sentiment** method of the Text Analytics service to discern the sentiment of two sentences.

In [8]:
body = {
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": "Wow! cognitive services are fantastic."
    },
    {
      "language": "en",
      "id": "2",
      "text": "I hate it when computers don't understand me."
    }
  ]
}

try:
    conn = http.client.HTTPSConnection(textAnalyticsURI)
    conn.request("POST", "/text/analytics/v2.0/sentiment?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read().decode("UTF-8")
    parsed = json.loads(data)
    for document in parsed['documents']:
        sentiment = "negative"
        if document["score"] >= 0.5:
            sentiment = "positive"
        print("Document:" + document["id"] + " = " + sentiment)
    conn.close()
    
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



Document:1 = positive
Document:2 = negative

## Working with Speech
So far, we've seen how analyze text; but increasingly AI systems enable humans to communicate with software services through speech recognition.

### Create a Bing Speech Service
The Microsoft Cognitive Services include the Bing Speech service, that can interpret spoken input from a microphone or audio file. Follow these steps to provision the Bing speech service:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ New**, and in the **AI + Cognitive Services** category, click **See all**.
4. In the list of cognitive services, click **Bing Speech API**.
5. In the **Bing Speech API** blade, click **Create**.
6. In the **Create** blade, enter the following details, and then click **Create**
  * **Name**: A unique name for your service.
  * **Subscription**: Your Azure subscription.
  * **Pricing tier**: Choose the F0 pricing tier.
  * **Resource Group**: Choose the existing resource group you used previously.
  * Read the notice about the use of your data, and select the checkbox.
7. Wait for the service to be created.
8. When deployment is complete, click **All Resources** and then click your Bing Speech service to open its blade.
10. In the blade for your Bing Speech service, click **Keys** and then copy **Key 1** to the clipboard and paste it into the **textKey** variable assignment value in the cell below.

In [10]:
speechKey = '<INSERT BING SPEECH API KEY HERE>'

### Install the SpeechRecognition Package
You can call the Bing Speech API by sending HTTP requests containing a JSON payload as you did for the Text Analytics API. However, the Bing Speech API is supported in a pre-built Python package named **SpeechRecognition**, which makes it easier to use. You can find out more about this package at https://pypi.python.org/pypi/SpeechRecognition.

We have attached the **SpeechRecognition** package to the cluster running this notebook

We'll use the following sample file for testing:<br>
https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/RainSpain.wav <br>
Right-click on above links and "Save Link As.." to download the audio files and listen to it

In [12]:
%sh
mkdir /dbfs/FileStore/media
curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/RainSpain.wav -o /dbfs/FileStore/media/RainSpain.wav
ls -lt /dbfs/FileStore/media

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 648k 100 648k 0 0 1091k 0 --:--:-- --:--:-- --:--:-- 1092k
total 0
-rw-r--r-- 1 root root 663648 Sep 4 18:45 RainSpain.wav

### Call the Bing Speech Service to Transcribe the Audio
Now that we know whats in the audio file, let's see how the Bing Speech API does!

In [14]:
import speech_recognition as sr

audioFile = '/dbfs/FileStore/media/RainSpain.wav'

# Read the audio file
r = sr.Recognizer(audioFile)
with sr.AudioFile() as source:
    audio = r.record(source)
    
# transcribe speech using the Bing Speech API
try:
    transcription = r.recognize_bing(audio, key=speechKey)
    print("Bing Transcription:")
    print('"' + transcription + '"')
    
except sr.UnknownValueError:
    print("The audio was unclear")
except sr.RequestError as e:
    print (e)
    print("Something went wrong :-(; {0}".format(e))


Here's what I heard:
"The rain in Spain stays mainly in the plain."

## Using the Language Understanding Intelligence Service (LUIS)
Increasingly, we expect computers to be able to use AI in order to understand spoken or typed commands in natural language. For example, we want to be able to say "switch on the light" or "put the light on", and have an AI-powered device understand the command and take appropriate action.

### Provision the Language Understanding Intelligence Service (LUIS)
The Microsoft cognitive services include the Language Understanding Intelligence Service (LUIS), which enables you to define *intents* that are applied to *entities* based on *utterances*.

To get started with LUIS, follow these steps to provision the service in your Azure subscription:
1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ New**, and in the **AI + Cognitive Services** category, click **See all**.
4. In the list of cognitive services, click **Language Understanding**.
5. In the **Language Understanding** blade, click **Create**.
6. In the **Create** blade, enter the following details, and then click **Create**
  * **Name**: A unique name for your service.
  * **Subscription**: Your Azure subscription.
  * **Location**: Choose a location in the US.
  * **Pricing tier**: Choose the F0 pricing tier.
  * **Resource Group**: Choose the existing resource group you used previously.
  * Read the notice about the use of your data, and select the checkbox.
7. Wait for the service to be created.

### Create a LUIS App
To implement natural language understanding with LUIS, you must first create an app; and then add intents, utterances, and entities to define the commands you want the app to understand.
1. Open a new browser tab and navigate to https://www.luis.ai/.
2. Sign in using the Microsoft account associated with your Azure subscription. If this is the first time you have signed into LUIS, you may need to grant the app some permissions to access your account details, and then fill in some information and accept the terms of use.
3. If a message prompting you to complete a tutorial in which you will create a *Scheduler* app is displayed, close it (you can complete this tutorial later - for now, we'll focus on a simpler example).
4. Click **New App** and create a new app with the following settings:
  * **Name**: Simple Home Automation
  * **Culture**: English
  * **Description**: A basic home automation example
5. In the pane on the left, click **Intents**. Then click **Add Prebuilt Domain Intent**, Search for **Home**, Add the following:
    * **HomeAutomation.TurnOff**
    * **HomeAutomation.TurnOn**
6. In the pane on the left, click **Entities**. Then click **Add Prebuilt Domain Entity**, Search for **Home**, Add the following:
    * **HomeAutomation.Device**
    * **HomeAutomation.Operation**
    * **HomeAutomation.Room**
7. At the top of the page, click **Train** to train the application
8. After the app has been trained, click **Test**, and then in the test pane, enter the following utterances and verify that they are correctly interpreted as commands for the *HomeAutomation.TurnOn* and *HomeAutomation.TurnOff* intents respectively:
    * *turn lights on*
    * *turn off the lights*
9. IAt the top of the page, click **Publish**. Then click **Publish to production slot**.
10. After the app has been published, note the **Endpoint** URL that is generated for your app - this includes the location where you provisioned the service, your app ID, and the key assigned to the app.

### Consume the LUIS App
Now that you have published your LUIS app, you can consume it from a client application by making HTTP requests that include a query string. The query will be used to identify the most likely intent, which will be returned to the calling client as in JSON response.

Modify the **endpointUrl** variable declaration in the cell below to reflect the endpoint URL for your app. Then run the cell, and enter a command when prompted to call your service and interpret the command. The JSON response is shown with an appropriate image for each command.

Try the following commands:
* *Switch on the light*
* *Turn on the light*
* *Turn off the light*
* *Could you switch on the lights on please?*

In [16]:
import requests
from io import BytesIO
import json 

# Set up API configuration
endpointUrl = "<INSERT LUIS ENDPOINT URL HERE WHICH INCLUDES THE KEY>"

# Test a command
command = 'Can you switch on the lights please?'

# Call the LUIS service and get the JSON response
endpoint = endpointUrl + command.replace(" ","+")
response = requests.get(endpoint)
data = json.loads(response.content.decode("UTF-8"))
print (data)

# Identify the top scoring intent
intent = data["topScoringIntent"]["intent"]
print ("Found Intent: " + intent)

{'intents': [{'intent': 'HomeAutomation.TurnOn', 'score': 0.9798689}, {'intent': 'None', 'score': 0.0185729563}, {'intent': 'HomeAutomation.TurnOff', 'score': 0.009096554}], 'entities': [{'score': 0.9591603, 'entity': 'lights', 'endIndex': 27, 'type': 'HomeAutomation.Device', 'startIndex': 22}], 'query': 'Can you switch on the lights please?', 'topScoringIntent': {'intent': 'HomeAutomation.TurnOn', 'score': 0.9798689}}
Found Intent: HomeAutomation.TurnOn

## Combining Speech Recognition and Natural Language Understanding
An obvious next step is to combine speech recognition with natural language understanding so that a spoken command can be interpreted and the appropriate action taken.

### Download Command Audio
Let's start by downloading and playing some spoken commands for our home automation system. <br>
https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOn.wav <br>
https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOff.wav <br>
Right-click on above links and "Save Link As.." to download the audio files and listen to them

In [18]:
%sh
# Get the "lights on" command
curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOn.wav -o /dbfs/FileStore/media/LightOn.wav
curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOff.wav -o /dbfs/FileStore/media/LightOff.wav
ls -lt /dbfs/FileStore/media

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 535k 0 1934 0 0 3806 0 0:02:24 --:--:-- 0:02:24 3799
100 535k 100 535k 0 0 828k 0 --:--:-- --:--:-- --:--:-- 828k
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 526k 100 526k 0 0 1147k 0 --:--:-- --:--:-- --:--:-- 1150k
total 0
-rw-r--r-- 1 root root 539452 Sep 4 18:53 LightOff.wav
-rw-r--r-- 1 root root 548368 Sep 4 18:53 LightOn.wav
-rw-r--r-- 1 root root 663648 Sep 4 18:45 RainSpain.wav

### Transcribe and Interpret the "Light On" Command
Now you can use the Bing Speech API to transcribe the command, and then use your LUIS app to interpret the command and take the appropriate action.

You should have already installed the **SpeechRecognition** package, set the **speechKey** for your Bing Speech service, and set the **apiUrl**, **appId**, and **appKey** for your LUIS app (if you have closed and re-opened the notebook, re-run the appropriate cells above to set these).

Run the cell below that to test the "Light On" command.

In [20]:
import speech_recognition as sr
from matplotlib.pyplot import imshow
from PIL import Image
import requests
from io import BytesIO
import json 

# Specify which audio file to use
audioFile = "/dbfs/FileStore/media/LightOff.wav"

# Read the audio file
r = sr.Recognizer()
with sr.AudioFile(audioFile) as source:
    audio = r.record(source)

try:
    # transcribe speech using the Bing Speech API
    transcription = r.recognize_bing(audio, key=speechKey)
    
    # Call the LUIS service and get the JSON response
    endpoint = endpointUrl + transcription.replace(" ","+")
    response = requests.get(endpoint)
    data = json.loads(response.content.decode("UTF-8"))

    # Identify the top scoring intent
    intent = data["topScoringIntent"]["intent"]
    print(data)
    print(intent)
    
except sr.UnknownValueError:
    print("Bing Speech could not understand audio")
except sr.RequestError as e:
    print (e)
    print("Could not request results from the Bing Speech service; {0}".format(e))


{'intents': [{'intent': 'HomeAutomation.TurnOff', 'score': 0.989046931}, {'intent': 'HomeAutomation.TurnOn', 'score': 0.016409345}, {'intent': 'None', 'score': 0.009934014}], 'entities': [{'score': 0.8938316, 'entity': 'light', 'endIndex': 13, 'type': 'HomeAutomation.Device', 'startIndex': 9}], 'query': 'Turn the light off.', 'topScoringIntent': {'intent': 'HomeAutomation.TurnOff', 'score': 0.989046931}}
HomeAutomation.TurnOff

### Transcribe and Interpret the "Light Off" Command
Modify the cell above to set the **audioFile** variable to the **LightOff.wav** audio file, and then run the cell again to test it